In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Conv2D, MaxPooling2D, BatchNormalization, \
    SeparableConv2D, GlobalAveragePooling2D
from keras.utils import to_categorical, Sequence
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras import initializers
from numpy.random import seed
import numpy as np
from keras import applications
import json
import random
import matplotlib.pyplot as plt
import cv2
import os
from glob import glob
import imgaug.augmenters as iaa
import imgaug as ia
import tensorflow as tf    

Using TensorFlow backend.
/home/yarosalv/Documents/Untitled/venv1/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/yarosalv/Documents/Untitled/venv1/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/yarosalv/Documents/Untitled/venv1/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home

In [2]:
labels = "./dataset/labels_for_test.json"
with open(labels) as label:
    data = json.load(label) 

In [3]:
def data_prepare(labels):
    data = {}
    for key in labels.keys():
        if labels[key] not in data:
            data.update({labels[key]: []})
        data[labels[key]].append(key)
    return data
data  = data_prepare(data)

In [4]:
max_len = 0
train_data = {}
val_data = {}
for i in data:
    if len(data[i]) > max_len:
        max_len = len(data[i])
    train_data.update({i: data[i][:int(0.9*len(data[i]))]})
    val_data.update({i: data[i][int(0.9*len(data[i])):]})

In [5]:
import pprint

In [6]:
img_width, img_height = 256, 256
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
seq = iaa.Sequential([
    iaa.Fliplr(0.8),
    sometimes(iaa.Affine(
    rotate=(-18, 18),
)),
])


for label in train_data:
    if not os.path.exists('dataset/learning_data/' + label):
        os.makedirs('dataset/learning_data/' + label)
        for i, img_name in enumerate(train_data[label]):
            image = cv2.imread('dataset/data_for_test/' +  img_name)
            gray = cv2.resize(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), (img_width, img_height))
            cv2.imwrite('dataset/learning_data/' + label + "/" + str(i) +".jpg", gray)
        for i in range(max_len-len(train_data[label])):
            image = cv2.imread('dataset/data_for_test/' + random.choice(train_data[label]))
            gray = cv2.resize(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), (img_width, img_height))
            images_aug = seq(images=gray)
            cv2.imwrite('dataset/learning_data/' + label + "/" + str(len(train_data[label]) + i) +".jpg", images_aug)
    else:
        print("Folder with train data {} prepared".format(label))

for label in val_data:
    if not os.path.exists('dataset/validation_data/' + label):
        os.makedirs('dataset/validation_data/' + label)
        for i, img_name in enumerate(val_data[label]):
            image = cv2.imread('dataset/data_for_test/' +  img_name)
            gray = cv2.resize(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), (img_width, img_height))
            cv2.imwrite('dataset/validation_data/' + label + "/" + str(i) +".jpg", gray)
    else:
        print("Folder with validation data {} prepared".format(label))

        
del(val_data)
del(train_data)
del(data)

Folder with train data classic_long prepared
Folder with train data goatee prepared
Folder with train data goatee_with_moustache prepared
Folder with train data classic_short prepared
Folder with train data moustache prepared
Folder with train data shaven prepared
Folder with train data stubble prepared
Folder with train data chin_curtain prepared
Folder with validation data classic_long prepared
Folder with validation data goatee prepared
Folder with validation data goatee_with_moustache prepared
Folder with validation data classic_short prepared
Folder with validation data moustache prepared
Folder with validation data shaven prepared
Folder with validation data stubble prepared
Folder with validation data chin_curtain prepared


In [2]:
img_width, img_height = 256, 256

In [3]:
train_data_dir = "dataset/learning_data/"
validation_data_dir = "dataset/validation_data/"
nb_validation_samples = 2
epochs = 50
batch_size = 8
base_model = applications.VGG16(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu',
          kernel_initializer=initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None))(x)

predictions = Dense(8, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')
nb_train_samples = train_generator.samples / batch_size
nb_validation_samples = validation_generator.samples / batch_size
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples)

Found 2408 images belonging to 8 classes.
Found 205 images belonging to 8 classes.
Epoch 1/50
301/301 [==============================] - 32s 106ms/step - loss: 2.0501 - acc: 0.1935 - val_loss: 1.9969 - val_acc: 0.1951
Epoch 2/50
301/301 [==============================] - 29s 97ms/step - loss: 1.8616 - acc: 0.2986 - val_loss: 1.9709 - val_acc: 0.2341
Epoch 3/50
301/301 [==============================] - 30s 101ms/step - loss: 1.7467 - acc: 0.3434 - val_loss: 2.0719 - val_acc: 0.2049
Epoch 4/50
301/301 [==============================] - 30s 100ms/step - loss: 1.6859 - acc: 0.3650 - val_loss: 1.8304 - val_acc: 0.3220
Epoch 5/50
301/301 [==============================] - 31s 103ms/step - loss: 1.5911 - acc: 0.4165 - val_loss: 1.9133 - val_acc: 0.2585
Epoch 6/50
301/301 [==============================] - 31s 103ms/step - loss: 1.5956 - acc: 0.4020 - val_loss: 1.8714 - val_acc: 0.3024
Epoch 7/50
301/301 [==============================] - 30s 101ms/step - loss: 1.5122 - acc: 0.4340 - val_loss